In [1]:
import spacy

In [ ]:
with open(input_filename, 'r') as f, open(output_filename, 'w') as g:
    for idx, line in enumerate(tqdm(f)):
        sentence  = line.strip()
        if len(sentence):
            valid_triplets = []
            for sent in nlp(sentence).sents:
                # Match
                for triplets in parse_sentence(sent.text, tokenizer, encoder, nlp, use_cuda=use_cuda):
                    valid_triplets.append(triplets)
            if len(valid_triplets) > 0:
                # Map
                mapped_triplets = []
                for triplet in valid_triplets:
                    head = triplet['h']
                    tail = triplet['t']
                    relations = triplet['r']
                    conf = triplet['c']
                    if conf < args.threshold:
                        continue
                    mapped_triplet = Map(head, relations, tail)
                    if 'h' in mapped_triplet:
                        mapped_triplet['c'] = conf
                        mapped_triplets.append(mapped_triplet)
                output = { 'line': idx, 'tri': deduplication(mapped_triplets) }

                if include_sentence:
                    output['sent'] = sentence
                if len(output['tri']) > 0:
                    g.write(json.dumps( output )+'\n')

In [23]:
class nlp:
    def __init__(self, sentence):
        self.text = sentence
        self.noun_chunks = []
        
def parse_sentence(*args, **kwargs):
    """ parse sentence """
    inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)
    triplets = []
    return triplets

def create_mapping(sentence, return_pt, nlp, tokenizer):
    inputs = None
    tokenid2word = None
    token2id = None
    noun_chunks = None
    return inputs, tokenid2word_mapping, token2id, noun_chunks

tokenizer = None
encoder = None

sentence = 'Bob Dylan är en gubbe som har skrivit musik.'

triplets = parse_sentence(sent.text, tokenizer, encoder, nlp)

In [22]:
def create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer):
    return None

def parse_sentence(sentence, tokenizer, encoder, nlp, use_cuda=True):
    ''' Parse stuff '''
    tokenizer_name = str(tokenizer.__str__)

    inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)

    with torch.no_grad():
        if use_cuda:
            for key in inputs.keys():
                inputs[key] = inputs[key].cuda()
        outputs = encoder(**inputs, output_attentions=True)
    trim = True
    if 'GPT2' in tokenizer_name:
        trim  = False

    '''
    Use average of last layer attention : page 6, section 3.1.2
    '''
    attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)

    merged_attention = compress_attention(attention, tokenid2word_mapping)
    attn_graph = build_graph(merged_attention)

    tail_head_pairs = []
    for head in noun_chunks:
        for tail in noun_chunks:
            if head != tail:
                tail_head_pairs.append((token2id[head], token2id[tail]))

    black_list_relation = set([ token2id[n]  for n in noun_chunks ])

    all_relation_pairs = []
    id2token = { value: key for key, value in token2id.items()}

    with Pool(10) as pool:
        params = [  ( pair[0], pair[1], attn_graph, max(tokenid2word_mapping), black_list_relation, ) for pair in tail_head_pairs]
        for output in pool.imap_unordered(bfs, params):
            if len(output):
                all_relation_pairs += [ (o, id2token) for o in output ]

    triplet_text = []
    with Pool(10, global_initializer, (nlp,)) as pool:
        for triplet in pool.imap_unordered(filter_relation_sets, all_relation_pairs):
            if len(triplet) > 0:
                triplet_text.append(triplet)
    return triplet_text

'Bob Dylan aer en gubbe som har skrivit musik.'

In [24]:
nlp = spacy.load('xx_ent_wiki_sm')
#nlp = spacy.load('da_core_news_md')

In [25]:
x = nlp(sentence)

In [26]:
list(x.noun_chunks)

[]